# Cleaning

## Import Data Frames

In [1]:
import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.metrics import make_scorer, f1_score, classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression
import shap

# Suppress warnings
warnings.filterwarnings("ignore")

zf = zipfile.ZipFile('SystemAccessLog_Start_Merged.csv.zip')
original_df = pd.read_csv(zf.open('SystemAccessLog_Start_Merged.csv'))

/Users/mayaghai/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Cleaning and Preprocessing

In [2]:
def check_group_consistency(dataframe: pd.DataFrame, group_by_col: str, check_col: str) -> pd.Series:
    """
    Checks for consistency within groups for a specific column.

    Calculates the number of unique values in 'check_col' for each group
    defined by 'group_by_col'. Returns a Series containing only those
    groups that have more than one unique value in 'check_col'.

    Args:
        dataframe: The pandas DataFrame to analyze.
        group_by_col: The name of the column to group the DataFrame by.
        check_col: The name of the column whose unique values within
                   each group should be counted.

    Returns:
        A pandas Series indexed by the unique values of 'group_by_col',
        showing the count (>1) of unique values in 'check_col' for
        groups that are inconsistent. Returns an empty Series if all
        groups are consistent for the checked column.

    Raises:
        KeyError: If group_by_col or check_col are not in the DataFrame's columns.
    """
    if group_by_col not in dataframe.columns:
        raise KeyError(f"Grouping column '{group_by_col}' not found in DataFrame.")
    if check_col not in dataframe.columns:
        raise KeyError(f"Checking column '{check_col}' not found in DataFrame.")

    # Group by the specified column and count unique values in the target column
    value_counts_per_group = dataframe.groupby(group_by_col)[check_col].nunique()

    # Filter to find groups with more than one unique value
    inconsistent_groups = value_counts_per_group[value_counts_per_group > 1]

    return inconsistent_groups

In [3]:
original_df.head()

,SessionId,Timestamp,Device,new_id,Session Time,AGE_GROUP_LONG_NAME,GENDER,ADMIT_TERM_CODE,APPL_STATUS_IN_CA_NAME,CURRENT_STAY_STATUS,PRIMARY_PROGRAM_CODE,SUCCESS_LEVEL,ACADEMIC_PERFORMANCE,term_end_date
0,122836922,2022-01-31,NaN,3c57943d195b8827,13:28:37.067Z,26 to 30,M,2022-01-01,Study Permit (student visa),Graduated - On Time,3825.0,Successful,AB - Good,2022-04-11
1,122836922,2022-01-31,NaN,3c57943d195b8827,15:39:16.723Z,26 to 30,M,2022-01-01,Study Permit (student visa),Graduated - On Time,3825.0,Successful,AB - Good,2022-04-11
2,122836922,2022-01-31,NaN,3c57943d195b8827,16:26:53.267Z,26 to 30,M,2022-01-01,Study Permit (student visa),Graduated - On Time,3825.0,Successful,AB - Good,2022-04-11
3,122836926,2022-01-31,NaN,43d2b11ec4c1a36e,13:28:38.187Z,26 to 30,F,2022-01-01,Study Permit (student visa),Graduated - On Time,1607.0,Successful,AB - Good,2022-04-11
4,122836926,2022-01-31,NaN,43d2b11ec4c1a36e,15:34:57.107Z,26 to 30,F,2022-01-01,Study Permit (student visa),Graduated - On Time,1607.0,Successful,AB - Good,2022-04-11


In [4]:
original_df.drop('Device',axis=1,inplace=True, errors='ignore') 

In [5]:
df = original_df.copy()

In [6]:
df['SessionId'] = df['SessionId'].astype(str)

In [7]:
print("\nMissing Value Percentage per Column:")
print((df.isnull().sum() / len(df)) * 100)


Missing Value Percentage per Column:


SessionId                 0.000000
Timestamp                 0.000000
new_id                    0.000000
Session Time              0.000000
AGE_GROUP_LONG_NAME       1.125323
GENDER                    0.116676
ADMIT_TERM_CODE           0.000000
APPL_STATUS_IN_CA_NAME    1.125323
CURRENT_STAY_STATUS       0.000000
PRIMARY_PROGRAM_CODE      0.000000
SUCCESS_LEVEL             0.000000
ACADEMIC_PERFORMANCE      0.000000
term_end_date             0.000000
dtype: float64


In [8]:
grouping_column = 'new_id'
column_to_check = 'ADMIT_TERM_CODE'

try:
    inconsistent_users = check_group_consistency(df, grouping_column, column_to_check)

    if not inconsistent_users.empty:
        print(f"'{grouping_column}' groups with more than one unique '{column_to_check}':")
        print(inconsistent_users)
    else:
        print(f"All '{grouping_column}' groups have a consistent '{column_to_check}'.")

except KeyError as e:
    print(f"Error: {e}")


'new_id' groups with more than one unique 'ADMIT_TERM_CODE':
new_id
003783fd936e5732    2
00bd23043142d0bf    2
00bfabd80ccd5ff2    2
00f419f65fffed5f    2
011f2b20e6581c60    2
                   ..
feb2581e4bbc5844    2
ff00d57e08dc525d    2
ff19fb22fa9b7ebc    2
ff3d6da65dfd3392    2
ffe9e357f3b82d20    2
Name: ADMIT_TERM_CODE, Length: 1392, dtype: int64


In [9]:
def handle_duplicate_admit_term_code(df):
    """
    Modifies 'new_id' for users with multiple unique 'ADMIT_TERM_CODE' values.
    
    For users with multiple ADMIT_TERM_CODEs, appends '_2', '_3', etc.
    to the 'new_id' for later ADMIT_TERM_CODEs.

    If multiple rows have the same ADMIT_TERM_CODE for a user, they share the same new_id.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The modified DataFrame.
    """
    df = df.copy()  # Don't modify original

    new_ids = []

    for name, group in df.groupby('new_id', sort=False):
        # Sort within group by ADMIT_TERM_CODE
        group_sorted = group.sort_values('ADMIT_TERM_CODE')

        # Track unique admit terms and assign suffix
        admit_terms = group_sorted['ADMIT_TERM_CODE'].unique()

        if len(admit_terms) > 1:
            # More than one admit term
            for i, term in enumerate(admit_terms):
                suffix = f"_{i+1}" if i > 0 else ""
                idxs = group_sorted[group_sorted['ADMIT_TERM_CODE'] == term].index
                for idx in idxs:
                    new_ids.append((idx, f"{name}{suffix}"))
        else:
            # Only one admit term → keep name
            for idx in group_sorted.index:
                new_ids.append((idx, name))

    # Assign new IDs
    new_id_map = dict(new_ids)
    df['new_id'] = df.index.map(new_id_map)

    return df


df = handle_duplicate_admit_term_code(df)

In [10]:
grouping_column = 'new_id'
column_to_check = 'ADMIT_TERM_CODE'

try:
    inconsistent_users = check_group_consistency(df, grouping_column, column_to_check)

    if not inconsistent_users.empty:
        print(f"'{grouping_column}' groups with more than one unique '{column_to_check}':")
        print(inconsistent_users)
    else:
        print(f"All '{grouping_column}' groups have a consistent '{column_to_check}'.")

except KeyError as e:
    print(f"Error: {e}")


All 'new_id' groups have a consistent 'ADMIT_TERM_CODE'.


In [11]:
def consolidate_success_level(group):
    if ('Successful' in group.values) or ('In progress' in group.values):
        return 'Successful'
    elif ('Unsuccessful' in group.values) or ('In Progress' in group.values):
        return 'Unsuccessful'

df['SUCCESS_LEVEL'] = df.groupby(grouping_column)['SUCCESS_LEVEL'].transform(consolidate_success_level)



In [12]:
COLUMN_TO_CONVERT = ['Timestamp', 'term_end_date']

# Check if the column exists and convert
for column in COLUMN_TO_CONVERT:
    if column in df.columns:
        df[column] = pd.to_datetime(df[column], errors='coerce')
        print(f"Converted '{column}' to datetime.")
    else:
        print(f"Column '{column}' not found in DataFrame.")

Converted 'Timestamp' to datetime.
Converted 'term_end_date' to datetime.


In [13]:
df['GENDER'].fillna('N', inplace=True)
df['AGE_GROUP_LONG_NAME'].fillna('Unknown', inplace=True)
df['APPL_STATUS_IN_CA_NAME'].fillna('Unknown', inplace=True)


### Aggregate Features for each Student

In [14]:
import pandas as pd
import numpy as np

print("Starting aggregation...")

# --- Prepare Data ---
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['activity_date'] = df['Timestamp'].dt.normalize()

# --- Active Days ---
active_days = df.groupby('new_id')['activity_date'].nunique().rename('unique_active_days')

# --- Daily & Weekly Standard Deviation ---
daily_counts = df.groupby(['new_id', 'activity_date']).size().rename('daily_log_count').reset_index()
daily_std = daily_counts.groupby('new_id')['daily_log_count'].std().fillna(0).rename('daily_log_std')

weekly_counts = df.groupby(['new_id', pd.Grouper(key='activity_date', freq='W-MON')]).size().rename('weekly_log_count').reset_index()
weekly_std = weekly_counts.groupby('new_id')['weekly_log_count'].std().fillna(0).rename('weekly_log_std')

# Combine std features
std_features = pd.concat([daily_std, weekly_std], axis=1)

# Longest Consecutive Active Days
def longest_streak(dates):
    if dates.empty:
        return 0
    sorted_dates = np.sort(dates.dt.normalize().unique())  # Ensures daily granularity
    sorted_dates_np = sorted_dates.astype('datetime64[D]')
    diffs = np.diff(sorted_dates_np).astype('timedelta64[D]').astype(int)
    streaks = np.split(sorted_dates_np, np.where(diffs != 1)[0] + 1)
    return max(len(streak) for streak in streaks)
# Calculate longest streak for each user
consecutive_days = df.groupby('new_id')['activity_date'].agg(longest_streak).rename('longest_consecutive_active_days')

# --- Rolling Average Trend Slope ---
def compute_rolling_trend(group):
    if len(group) < 2:
        return 0.0
    x = pd.to_datetime(group['activity_date']).view(np.int64) // 10**9  # Use .view instead of deprecated .astype
    y = group['daily_log_count'].to_numpy()
    try:
        slope, _ = np.polyfit(x, y, 1)
        return slope
    except np.linalg.LinAlgError:
        return 0.0

rolling_trends = (
    daily_counts
    .sort_values(['new_id', 'activity_date'])
    .groupby('new_id', group_keys=False)
    .apply(compute_rolling_trend)
    .rename('rolling_avg_trend_slope')
)

# --- Log Deviation from Program Average ---
user_logs = df.groupby(['new_id', 'PRIMARY_PROGRAM_CODE']).size().rename('user_log_count').reset_index()
program_avg = user_logs.groupby('PRIMARY_PROGRAM_CODE')['user_log_count'].mean().rename('program_avg_logs').reset_index()
log_dev = (
    user_logs.merge(program_avg, on='PRIMARY_PROGRAM_CODE')
    .assign(log_deviation_from_program=lambda x: x['user_log_count'] - x['program_avg_logs'])
    .set_index('new_id')['log_deviation_from_program']
)

# --- Rolling Daily Average (7-day) ---
rolling_avg_7d = (
    daily_counts
    .sort_values(['new_id', 'activity_date'])
    .groupby('new_id')['daily_log_count']
    .transform(lambda x: x.rolling(7, min_periods=1).mean())
)

avg_rolling_7d = (
    daily_counts.assign(rolling_avg_7d=rolling_avg_7d)
    .groupby('new_id')['rolling_avg_7d']
    .mean()
    .rename('rolling_7d_avg_logs')
)

# --- First Week Stats ---
first_timestamp = df.groupby('new_id')['Timestamp'].min()
df = df.join(first_timestamp.rename('first_timestamp'), on='new_id')
df['within_first_week'] = (df['Timestamp'] - df['first_timestamp']) < pd.Timedelta(weeks=1)

log_count_first = df[df['within_first_week']].groupby('new_id').size().rename('LOG_COUNT_first_week')

log_std_first = (
    df[df['within_first_week']]
    .groupby(['new_id', 'activity_date']).size()
    .groupby('new_id').std().fillna(0)
    .rename('LOG_STD_first_week')
)

# --- Main Aggregation ---
agg_map = {
    'SUCCESS_LEVEL': 'last',
    'AGE_GROUP_LONG_NAME': 'first',
    'GENDER': 'first',
    'ADMIT_TERM_CODE': 'first',
    'APPL_STATUS_IN_CA_NAME': 'first',
    'CURRENT_STAY_STATUS': 'last',
    'PRIMARY_PROGRAM_CODE': 'first',
    'ACADEMIC_PERFORMANCE': 'first',
    'SessionId': 'count',
    'Timestamp': ['min', 'max']
}

df_user = df.groupby('new_id').agg(agg_map)
df_user.columns = ['_'.join(filter(None, col)).strip('_') for col in df_user.columns]

# --- Join Engineered Features ---
df_user = df_user.join([
    active_days,
    log_dev,
    avg_rolling_7d,
    std_features,
    consecutive_days,
    rolling_trends,
    log_count_first,
    log_std_first
])

# --- Session Span ---
df_user['session_span_days'] = (df_user['Timestamp_max'] - df_user['Timestamp_min']).dt.total_seconds() / (60*60*24)

# --- Activity Delay ---
def calculate_activity_delay(row):
    try:
        admit_date = pd.to_datetime(row['ADMIT_TERM_CODE_first'], errors='coerce')
        if pd.isnull(admit_date):
            return None
        delay = row['Timestamp_min'] - admit_date
        return delay.total_seconds() / (60 * 60 * 24)
    except Exception:
        return None

df_user['activity_delay_days'] = df_user.apply(calculate_activity_delay, axis=1)

print("Aggregation completed.")
df_user.info()


Starting aggregation...
Aggregation completed.
<class 'pandas.core.frame.DataFrame'>
Index: 38494 entries, 0000cf80a3f3206a to fffdecaa52500c60
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   SUCCESS_LEVEL_last               38494 non-null  object        
 1   AGE_GROUP_LONG_NAME_first        38494 non-null  object        
 2   GENDER_first                     38494 non-null  object        
 3   ADMIT_TERM_CODE_first            38494 non-null  object        
 4   APPL_STATUS_IN_CA_NAME_first     38494 non-null  object        
 5   CURRENT_STAY_STATUS_last         38494 non-null  object        
 6   PRIMARY_PROGRAM_CODE_first       38494 non-null  float64       
 7   ACADEMIC_PERFORMANCE_first       38494 non-null  object        
 8   SessionId_count                  38494 non-null  int64         
 9   Timestamp_min                    38494 non-null  datetime64

### Map Ordinal Categories

In [15]:
df_user['AGE_GROUP_LONG_NAME_first'].unique()

array(['36 to 40', '0 to 18', '31 to 35', '26 to 30', '21 to 25',
       '41 to 50', '19 to 20', '51 to 60', 'Unknown', '66+', '61 to 65'],
      dtype=object)

In [16]:
df_user['ACADEMIC_PERFORMANCE_first'].unique()

array(['AB - Good', 'DF - Poor', 'C - Satisfactory', 'ZZ - Unknown'],
      dtype=object)

In [17]:
# Define the ordinal encodings in a dictionary of format:
# {column_name: ([ordered_categories], new_column_name)}
ordinal_encodings = {
    'AGE_GROUP_LONG_NAME_first': (
        ['0 to 18', '19 to 20', '21 to 25', '26 to 30', '31 to 35',
         '36 to 40', '41 to 50', '51 to 60', '61 to 65', '66+','Unknown'],
        # Note: 'Unknown' is added to the end of the list, making it the highest ordinal value. This is a design choice.
        'AGE_GROUP_ordinal'
    ),
    'ACADEMIC_PERFORMANCE_first': (
        ['DF - Poor', 'C - Satisfactory', 'AB - Good'],
        'ACADEMIC_PERFORMANCE_ordinal'
    )
}

for col, (ordered_values, new_col) in ordinal_encodings.items():
    if col in df_user.columns:
        # Create mapping
        mapping = {k: i for i, k in enumerate(ordered_values)}
        if col == 'ACADEMIC_PERFORMANCE_first':
            mapping['ZZ - Unknown'] = 0  # Assign 'ZZ - Unknown' to 0
        #

        # Apply and create new column
        df_user[new_col] = df_user[col].map(mapping)
        df_user.drop(columns=col, inplace=True)  # Drop original string column

        # Log results
        print(f"Encoded '{col}' → '{new_col}'")
        print(df_user[new_col].value_counts(dropna=False).sort_index())
    else:
        print(f"Column '{col}' not found.")


Encoded 'AGE_GROUP_LONG_NAME_first' → 'AGE_GROUP_ordinal'
AGE_GROUP_ordinal
0      4316
1      6905
2     12173
3      6881
4      3245
5      2198
6      1944
7       351
8        32
9        11
10      438
Name: count, dtype: int64
Encoded 'ACADEMIC_PERFORMANCE_first' → 'ACADEMIC_PERFORMANCE_ordinal'
ACADEMIC_PERFORMANCE_ordinal
0     5787
1     8139
2    24568
Name: count, dtype: int64


In [18]:
df_user.reset_index()

,new_id,SUCCESS_LEVEL_last,GENDER_first,ADMIT_TERM_CODE_first,APPL_STATUS_IN_CA_NAME_first,CURRENT_STAY_STATUS_last,PRIMARY_PROGRAM_CODE_first,SessionId_count,Timestamp_min,Timestamp_max,...,daily_log_std,weekly_log_std,longest_consecutive_active_days,rolling_avg_trend_slope,LOG_COUNT_first_week,LOG_STD_first_week,session_span_days,activity_delay_days,AGE_GROUP_ordinal,ACADEMIC_PERFORMANCE_ordinal
0,0000cf80a3f3206a,Successful,F,2022-09-01,Study Permit (student visa),Graduated - On Time,1207.0,524,2022-09-01,2022-12-10,...,3.319196,13.095619,41,2.265733e-07,12,1.414214,100.0,0.0,5,2
1,00024c40cef486cc,Unsuccessful,M,2022-09-01,Canadian Citizen,Left College - Enrolled 2 Terms,3703.0,157,2022-09-06,2022-12-10,...,2.162455,5.820105,7,3.183529e-08,17,2.073644,95.0,5.0,0,0
2,0002d2ebd89ea8cd,Successful,M,2022-09-01,Study Permit (student visa),Graduated - On Time,3776.0,437,2022-09-06,2022-12-10,...,2.407064,7.505676,33,2.731579e-07,25,2.225395,95.0,5.0,4,2
3,000698ff5c581e09,Successful,F,2022-09-01,Study Permit (student visa),Graduated - On Time,2890.0,302,2022-09-01,2022-12-10,...,2.811847,10.439394,35,9.430580e-08,3,0.000000,100.0,0.0,3,2
4,000915a44be41906,Unsuccessful,F,2022-09-01,Study Permit (student visa),Stepped Out - 2 Terms Away,3109.0,431,2022-09-06,2022-12-10,...,3.340034,12.166654,21,-9.588078e-08,45,3.779645,95.0,5.0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38489,ffed705b05a2a6fa,Successful,F,2022-09-01,Study Permit (student visa),Graduated - On Time,2870.0,330,2022-09-06,2022-12-10,...,2.939797,8.308608,24,-1.454189e-07,18,1.290994,95.0,5.0,3,2
38490,fff05347678bc430,Unsuccessful,F,2023-05-01,Study Permit (student visa),Stepped Out - 1 Term Away,1824.0,370,2023-05-06,2023-08-09,...,3.681548,13.264704,19,-1.515316e-07,22,3.049590,95.0,5.0,1,1
38491,fff0860d19afbee5,Successful,F,2023-09-01,Study Permit (student visa),Graduated - On Time,2860.0,817,2023-09-01,2023-12-10,...,4.146165,18.384258,97,-1.185232e-07,35,6.000000,100.0,0.0,2,2
38492,fffd021c310b2e45,Successful,F,2022-05-01,Study Permit (student visa),Graduated - On Time,2805.0,215,2022-05-12,2022-08-09,...,2.391998,9.026518,10,2.677611e-08,14,1.643168,89.0,11.0,1,1


### Check for any Remaining Missing Values

In [19]:
# Fill missing values in 'APPL_STATUS_IN_CA_NAME_first' with 'Unknown'
df_user.fillna({'APPL_STATUS_IN_CA_NAME_first': 'Unknown'}, inplace=True)

In [20]:
# Fill NaN with a string (e.g., 'Missing')
for col in df_user.columns:
    if df_user[col].dtype == 'object':
        # Fill NaN with a specific string
        df_user[col] = df_user[col].fillna('Unknown')

In [21]:
for col in df_user.columns:
    print(f"\nValue Counts for {col}:")
    print(df_user[col].value_counts(normalize=False))
    print(f"Missing in {col}: {df_user[col].isnull().sum()}")


Value Counts for SUCCESS_LEVEL_last:
SUCCESS_LEVEL_last
Unsuccessful    19439
Successful      19055
Name: count, dtype: int64
Missing in SUCCESS_LEVEL_last: 0

Value Counts for GENDER_first:
GENDER_first
F    19388
M    18975
N      131
Name: count, dtype: int64
Missing in GENDER_first: 0

Value Counts for ADMIT_TERM_CODE_first:
ADMIT_TERM_CODE_first
2022-09-01    10721
2023-09-01     9517
2023-01-01     7504
2022-01-01     5220
2023-05-01     2968
2022-05-01     2555
2021-09-01        8
2021-05-01        1
Name: count, dtype: int64
Missing in ADMIT_TERM_CODE_first: 0

Value Counts for APPL_STATUS_IN_CA_NAME_first:
APPL_STATUS_IN_CA_NAME_first
Study Permit (student visa)                           24391
Canadian Citizen                                      10792
Permanent Resident or Landed Immigrant                 2543
Unknown                                                 446
Conv Refugee/Protected person/Work Permit/Diplomat      277
Other                                          

# Drill down on Program Codes

In [22]:
program_codes = pd.read_csv("program_codes.csv")

In [23]:
program_codes.head(5)

,PRIMARY_PROGRAM_CODE,PROGRAM_CAMPUS_CODE,PROGRAM_CREDENTIAL_CODE,PROGRAM_CREDENTIAL_NAME,PROGRAM_LONG_NAME,SCHOOL_CODE,CLUSTER_LONG_NAME
0,9111,HPC,OCCT,Ontario College Certificate,Personal Support Worker,CH,SCHS-PRACTICAL NURSING AND PSW
1,2870,PRO,OCGC,Ontario College Graduate Cert,Paralegal,BU,BUS-LEGAL AND ADMIN STUDIES
2,9320,PRO,OCDP,Ontario College Diploma,Fitness and Health Promotion,CH,SCHS-HEALTH AND WELLNESS STUDIES
3,2003,OLP,OCDP,Ontario College Diploma,Office Administration - Health Services,BU,BUS-LEGAL AND ADMIN STUDIES
4,2001,OLP,OCCT,Ontario College Certificate,Office Administration - General,BU,BUS-LEGAL AND ADMIN STUDIES


In [24]:
program_codes['CLUSTER_LONG_NAME'].unique()

array(['SCHS-PRACTICAL NURSING AND PSW', 'BUS-LEGAL AND ADMIN STUDIES',
       'SCHS-HEALTH AND WELLNESS STUDIES',
       'SETAS-AD MANUFCT & AUTO TECH (AMAT)',
       'BUS-MARKETING&ENTREPRENEURSHIP',
       'BUS-ACCOUNTING & FINANCIAL SERVICES',
       'SHTCA-HOSPITALITY MANAGEMENT',
       'SCHS-REGISTERED  NURSING PROGRAMS',
       'SHTCA-FOOD AND TOURISM STUDIES',
       'SETAS-APPL BIO & ENVIRN SCIENCE', 'SCHS-CHILD STUDIES',
       'CMAD-COMM, MEDIA & DESIGN', 'SOT-AVIATION',
       'SCHS-EMERG MGT AND PUBLIC SAFETY',
       'BUS-BUSINESS & MANAGEMENT STUDIES',
       'SETAS-INFORM & COMMUNICATION TECH',
       'SELS-SCHOOL OF ENG.&LIBERAL STUDIES', 'SCHS-COMMUNITY SERVICES',
       nan, 'BUS-INTERNATIONAL BUSINESS & LOGIST', 'SETAS - SDRE',
       'SOT - AUTOMOTIVE & MOTORCYCLE', 'SOT- MOTIVE POWER',
       'CMAD-ARTS & DESIGN', 'SCHS-RESTRICTED FUNDS',
       'SOT-AIRFRAME ASSEMBLY', 'SOT-CO-OP DIPLOMA PROGRAMS',
       'SETAS-CO-OP DIPLOMA PROGRAMS', 'SETAS-RESTRICTED FUNDS',

In [25]:
# Define mapping dictionary
program_category_map = {
    'SCHS-PRACTICAL NURSING AND PSW': 'Community & Health Studies',
    'SCHS-HEALTH AND WELLNESS STUDIES': 'Community & Health Studies',
    'SCHS-REGISTERED  NURSING PROGRAMS': 'Community & Health Studies',
    'SCHS-CHILD STUDIES': 'Community & Health Studies',
    'SCHS-EMERG MGT AND PUBLIC SAFETY': 'Community & Health Studies',
    'SCHS-COMMUNITY SERVICES': 'Community & Health Studies',
    'SCHS-RESTRICTED FUNDS': 'Community & Health Studies',

    'BUS-LEGAL AND ADMIN STUDIES': 'Business School',
    'BUS-MARKETING&ENTREPRENEURSHIP': 'Business School',
    'BUS-ACCOUNTING & FINANCIAL SERVICES': 'Business School',
    'BUS-BUSINESS & MANAGEMENT STUDIES': 'Business School',
    'BUS-INTERNATIONAL BUSINESS & LOGIST': 'Business School',

    'SETAS-AD MANUFCT & AUTO TECH (AMAT)': 'Engineering Technology & Applied Science',
    'SETAS-APPL BIO & ENVIRN SCIENCE': 'Engineering Technology & Applied Science',
    'SETAS-INFORM & COMMUNICATION TECH': 'Engineering Technology & Applied Science',
    'SETAS - SDRE': 'Engineering Technology & Applied Science',
    'SETAS-CO-OP DIPLOMA PROGRAMS': 'Engineering Technology & Applied Science',
    'SETAS-RESTRICTED FUNDS': 'Engineering Technology & Applied Science',

    'SHTCA-HOSPITALITY MANAGEMENT': 'Hospitality, Tourism & Culinary Arts',
    'SHTCA-FOOD AND TOURISM STUDIES': 'Hospitality, Tourism & Culinary Arts',

    'CMAD-COMM, MEDIA & DESIGN': 'Communications, Media, Arts & Design',
    'CMAD-ARTS & DESIGN': 'Communications, Media, Arts & Design',

    'SOT-AVIATION': 'Transportation',
    'SOT - AUTOMOTIVE & MOTORCYCLE': 'Transportation',
    'SOT- MOTIVE POWER': 'Transportation',
    'SOT-AIRFRAME ASSEMBLY': 'Transportation',
    'SOT-CO-OP DIPLOMA PROGRAMS': 'Transportation',

    'SELS-SCHOOL OF ENG.&LIBERAL STUDIES': 'English & Liberal Studies',

    'CORPORATE CONTROL': 'Other'
}

# Apply to DataFrame
program_codes['program_category'] = program_codes['CLUSTER_LONG_NAME'].map(program_category_map)


In [26]:
# Map null values to Business School after inspecting rows 
program_codes.loc[program_codes['program_category'].isnull(), 'program_category'] = 'Business School'
# Check for any remaining null values
remaining_nulls = program_codes[program_codes['program_category'].isnull()]
if not remaining_nulls.empty:
    print("Remaining null values in 'program_category':")
    print(remaining_nulls)
# Check distribution of program categories and check for missing values
print("\nProgram Category Distribution:")
print(program_codes['program_category'].value_counts(normalize=True))
print("\nMissing Values in Program Category:")
print(program_codes['program_category'].isnull().sum())


Program Category Distribution:
program_category
Engineering Technology & Applied Science    0.327869
Business School                             0.266979
Community & Health Studies                  0.112412
Hospitality, Tourism & Culinary Arts        0.100703
Communications, Media, Arts & Design        0.100703
Transportation                              0.067916
English & Liberal Studies                   0.021077
Other                                       0.002342
Name: proportion, dtype: float64

Missing Values in Program Category:
0


In [27]:
credential_mapping_reduced = {
    'Ontario College Certificate': 'Certificate',
    'Ontario College Graduate Cert': 'Certficate',
    'Ontario College Diploma': 'Diploma',
    'Ontario College Adv. Diploma': 'Diploma',
    'Degree': 'University',
    'Bachelor of Science Nursing': 'University',
    'Undeclared': 'Undeclared'
}

program_codes['PROGRAM_CREDENTIAL_mapped'] = program_codes['PROGRAM_CREDENTIAL_NAME'].map(credential_mapping_reduced)

In [28]:
program_codes_to_merge = program_codes[['PRIMARY_PROGRAM_CODE', 'program_category', 'PROGRAM_CREDENTIAL_mapped']]
# Rename the column in program_codes_to_merge to match df_user
program_codes_to_merge.rename(columns={'PRIMARY_PROGRAM_CODE': 'PRIMARY_PROGRAM_CODE_first'}, inplace=True)

In [34]:
# Merge the DataFrames
df_user_merged = df_user.reset_index().merge(program_codes_to_merge, on='PRIMARY_PROGRAM_CODE_first', how='left')

In [35]:
df_user_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38494 entries, 0 to 38493
Data columns (total 25 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   new_id                           38494 non-null  object        
 1   SUCCESS_LEVEL_last               38494 non-null  object        
 2   GENDER_first                     38494 non-null  object        
 3   ADMIT_TERM_CODE_first            38494 non-null  object        
 4   APPL_STATUS_IN_CA_NAME_first     38494 non-null  object        
 5   CURRENT_STAY_STATUS_last         38494 non-null  object        
 6   PRIMARY_PROGRAM_CODE_first       38494 non-null  float64       
 7   SessionId_count                  38494 non-null  int64         
 8   Timestamp_min                    38494 non-null  datetime64[ns]
 9   Timestamp_max                    38494 non-null  datetime64[ns]
 10  unique_active_days               38494 non-null  int64    

In [37]:
# Write the DataFrame to a CSV file
df_user_merged.to_csv('df_user_aggregated.csv', index=False)